# Sattler

Dataset: https://www.dropbox.com/sh/4adrgjsee60vcvj/AADJ-hbes1uHg3FE0et69sy5a?dl=1

Say we have these files...

In [9]:
from pathlib import Path
from pprint import pprint

# we've symlinked the example data folder to the cwd for this example
base_path = Path().cwd()  / '2021-02-26_17-19-10_mouse-0232'

data_files = [str(path.relative_to(base_path)) for path in base_path.glob("**/*")]
pprint(sorted(data_files))

['.DS_Store',
 '2021-02-26_17-19-12_mouse-0232',
 '2021-02-26_17-19-12_mouse-0232/103_ADC1.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_ADC2.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_ADC3.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_ADC4.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_ADC5.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_ADC6.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_ADC7.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_ADC8.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_AUX1.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_AUX2.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_AUX3.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_CH1.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_CH10.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_CH11.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_CH12.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_CH13.continuous',
 '2021-02-26_17-19-12_mouse-0232/103_CH14.continuous',
 '2021-02-26_17-1

Which compose a dataset of

* Continuous extracellular ephys data recorded by open ephys
* Spikes sorted by Kilosort
* Stimulus information from some custom behavioral software
* Raw video of the behaving animal.
